In [1]:
import numpy as np
import pandas as pd
from chromatography import *
from separation_utility import *
from torch import optim, tensor
import torch.nn as nn
import matplotlib.pyplot as plt
import time

In [2]:
alists = []
alists.append(pd.read_csv('../data/GilarSample.csv'))
alists.append(pd.read_csv('../data/Alizarin.csv'))
alists.append(pd.read_csv('../data/Peterpeptides.csv'))
alists.append(pd.read_csv('../data/Roca.csv'))
alists.append(pd.read_csv('../data/Peter32.csv'))
alists.append(pd.read_csv('../data/Eosin.csv'))
alists.append(pd.read_csv('../data/Controlmix2.csv'))
alists.append(pd.read_csv('../data/Gooding.csv'))
# GilarSample - 8 analytes
# Peterpeptides - 32 analytes
# Roca - 14 analytes
# Peter32 - 32 analytes
# Eosin - 20 analytes
# Alizarin - 16 analytes
# Controlmix2 - 17 analytes
# Gooding - 872 analytes

In [ ]:
def loss_field(exp, taus, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], taus)
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses

def average_over_equal_intervals(arr, interval):
    return np.mean(arr.reshape(-1, interval), axis=1)

### Experiment: Performance vs n_steps

In [ ]:
# Parameters
all_analytes = pd.concat(alists, sort=True).reset_index()[['k0', 'S', 'lnk0']]

kwargs = {
    'num_episodes' : 25_000, 
    'sample_size' : 10,
    'batch_size' : 1, 
    'lr' : .05, 
    'optim' : torch.optim.SGD,
    'lr_decay_factor' : 0.75,
    'lr_milestones' : 5000,
    'print_every' : 25_001,
    'baseline' : .55,
    'max_norm' : 1.5,
    'max_rand_analytes' : 40,
    'min_rand_analytes' : 8,
    'rand_prob' : 1.,
    'h' : 0.001,
    'run_time' : 1.
}
N = 7
M = 15

losses_50_50 = np.zeros((N, M, kwargs['num_episodes']))
test_losses_50_50 = np.zeros((N, M, kwargs['num_episodes']))
losses_100 = np.zeros((N, M, kwargs['num_episodes']))

for n in range(0, N):
    print(n)
    delta_taus = np.ones(n + 1) * 1/(n + 1)
    
    for i in range(M):
        alist_train = all_analytes.sample(frac=0.5)
        alist_test = all_analytes.loc[lambda a: ~a.index.isin(alist_train.index.values)]
        print(f"  {i}")
        #Policies
        pol_50_50 = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
            ),
            rho = nn.Sequential(
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                Rho(n_steps=len(delta_taus), hidden=5, in_dim=5, sigma_max=.3, sigma_min=.01),
            )
        )
        pol_100 = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
            ),
            rho = nn.Sequential(
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                Rho(n_steps=len(delta_taus), hidden=5, in_dim=5, sigma_max=.3, sigma_min=.01),
            )
        )
        # Run Exp
        loss, loss_test = reinforce_gen(
            alists = [alist_train], 
            test_alist = alist_test,
            policy = pol_50_50, 
            delta_taus = delta_taus, 
            **kwargs
        )
        loss_100, _ = reinforce_gen(
            alists = [all_analytes], 
            test_alist = None,
            policy = pol_100, 
            delta_taus = delta_taus, 
            **kwargs
        )
        
        losses_50_50[n,i] = loss
        test_losses_50_50[n,i] = loss_test
        losses_100[n,i] = loss_100


In [ ]:
#np.savez_compressed("../results/general_perf_vs_n_steps", losses_50_50=losses_50_50, test_losses_50_50=test_losses_50_50, losses_100=losses_100)
np.savez_compressed("../results/general_perf_vs_n_steps_losses_50", losses_50_50=losses_50_50)
np.savez_compressed("../results/general_perf_vs_n_steps_test_losses_50_50", test_losses_50_50=test_losses_50_50)
np.savez_compressed("../results/general_perf_vs_n_steps_losses_100", losses_100=losses_100)

### Performance vs number of analytes

In [ ]:
# Parameters
all_analytes = pd.concat(alists, sort=True).reset_index()[['k0', 'S', 'lnk0']]

kwargs = {
    'num_episodes' : 25_000, 
    'sample_size' : 10,
    'batch_size' : 1, 
    'lr' : .05, 
    'optim' : torch.optim.SGD,
    'lr_decay_factor' : 0.75,
    'lr_milestones' : 5000,
    'print_every' : 25_001,
    'baseline' : .55,
    'max_norm' : 1.5,
    'rand_prob' : 1.,
    'h' : 0.001,
    'run_time' : 1.
}
N = 5
M = 30

losses_50_50 = np.zeros((N, M, kwargs['num_episodes']))
test_losses_50_50 = np.zeros((N, M, kwargs['num_episodes']))
losses_100 = np.zeros((N, M, kwargs['num_episodes']))


delta_taus = np.ones(10) * 1/(10)
for n in range(N):
    for i in range(M):
        alist_train = all_analytes.sample(frac=0.5)
        alist_test = all_analytes.loc[lambda a: ~a.index.isin(alist_train.index.values)]
        print(f"  {i}")
        #Policies
        pol_50_50 = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
            ),
            rho = nn.Sequential(
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                Rho(n_steps=len(delta_taus), hidden=5, in_dim=5, sigma_max=.3, sigma_min=.01),
            )
        )
        pol_100 = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
            ),
            rho = nn.Sequential(
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                Rho(n_steps=len(delta_taus), hidden=5, in_dim=5, sigma_max=.3, sigma_min=.01),
            )
        )
        # Run Exp
        loss, loss_test = reinforce_gen(
            alists = [alist_train], 
            test_alist = alist_test,
            policy = pol_50_50, 
            delta_taus = delta_taus,
            min_rand_analytes = 8 * (n + 1),
            max_rand_analytes = 8 * (n + 1),
            **kwargs
        )
        loss_100, _ = reinforce_gen(
            alists = [all_analytes], 
            test_alist = None,
            policy = pol_100, 
            delta_taus = delta_taus,
            min_rand_analytes = 8 * (n + 1),
            max_rand_analytes = 8 * (n + 1),
            **kwargs
        )

        losses_50_50[n,i] = loss
        test_losses_50_50[n,i] = loss_test
        losses_100[n,i] = loss_100


In [ ]:
np.savez_compressed("../results/general_perf_vs_nr_analytes_losses_50", losses_50_50=losses_50_50)
np.savez_compressed("../results/general_perf_vs_nr_analytes_test_losses_50_50", test_losses_50_50=test_losses_50_50)
np.savez_compressed("../results/general_perf_vs_nr_analytes_losses_100", losses_100=losses_100)

### Performance vs architecture DeepSet(phi)

In [4]:
# Parameters
all_analytes = pd.concat(alists, sort=True).reset_index()[['k0', 'S', 'lnk0']]
activations = [nn.ELU, nn.ReLU, nn.Tanh]
width = [5, 10, 20]
kwargs = {
    'num_episodes' : 1, #25_000, 
    'sample_size' : 10,
    'batch_size' : 1, 
    'lr' : .05, 
    'optim' : torch.optim.SGD,
    'lr_decay_factor' : 0.75,
    'lr_milestones' : 5000,
    'print_every' : 25_001,
    'baseline' : .55,
    'max_norm' : 1.5,
    'max_rand_analytes' : 40,
    'min_rand_analytes' : 8,
    'rand_prob' : 1.,
    'h' : 0.001,
    'run_time' : 1.
}
N = 9
M = 20

losses_deep_set = np.zeros((N, M, kwargs['num_episodes']))
test_losses_deep_set = np.zeros((N, M, kwargs['num_episodes']))

delta_taus = np.ones(10) * 1/(10)
for i in range(N):
    print(f"{i}")
    for m in range(M):
        alist_train = all_analytes.sample(frac=0.5)
        alist_test = all_analytes.loc[lambda a: ~a.index.isin(alist_train.index.values)]
        print(f"  {m}")
        #Policies
        pol_50_50 = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, width[i % 3]),
                activations[i // 3](),
                PermEqui2_max(width[i % 3], width[i % 3]),
                activations[i // 3](),
                PermEqui2_max(width[i % 3], width[i % 3]),
                activations[i // 3](),
            ),
            rho = nn.Sequential(
                nn.Linear(width[i % 3], 5),
                nn.ELU(inplace=True),
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                Rho(n_steps=len(delta_taus), hidden=5, in_dim=5, sigma_max=.3, sigma_min=.01),
            )
        )
        
        # Run Exp
        loss, loss_test = reinforce_gen(
            alists = [alist_train], 
            test_alist = alist_test,
            policy = pol_50_50, 
            delta_taus = delta_taus, 
            **kwargs
        )

        losses_deep_set[i, m] = loss
        test_losses_deep_set[i, m] = loss_test

0
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
  10
  11
  12
  13
  14
  15
  16
  17
  18
  19
PolicyGeneral(
  (phi): Sequential(
    (0): PermEqui2_max(
      (Gamma): Linear(in_features=2, out_features=5, bias=True)
      (Lambda): Linear(in_features=2, out_features=5, bias=False)
    )
    (1): ELU(alpha=1.0)
    (2): PermEqui2_max(
      (Gamma): Linear(in_features=5, out_features=5, bias=True)
      (Lambda): Linear(in_features=5, out_features=5, bias=False)
    )
    (3): ELU(alpha=1.0)
    (4): PermEqui2_max(
      (Gamma): Linear(in_features=5, out_features=5, bias=True)
      (Lambda): Linear(in_features=5, out_features=5, bias=False)
    )
    (5): ELU(alpha=1.0)
  )
  (rho): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): ELU(alpha=1.0, inplace=True)
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ELU(alpha=1.0, inplace=True)
    (4): Rho(
      (sig): Sigmoid()
      (fc_mu_1): Linear(in_features=5, out_features=5, bias=True)


/home/stefan/Thesis/code/chromatography.py:251: RuntimeWarning: divide by zero encountered in true_divide
  return delta_tau_phi * (1 + self.k(phi)) / self.k(phi)


  11
  12
  13
  14
  15
  16
  17
  18
  19
PolicyGeneral(
  (phi): Sequential(
    (0): PermEqui2_max(
      (Gamma): Linear(in_features=2, out_features=10, bias=True)
      (Lambda): Linear(in_features=2, out_features=10, bias=False)
    )
    (1): ELU(alpha=1.0)
    (2): PermEqui2_max(
      (Gamma): Linear(in_features=10, out_features=10, bias=True)
      (Lambda): Linear(in_features=10, out_features=10, bias=False)
    )
    (3): ELU(alpha=1.0)
    (4): PermEqui2_max(
      (Gamma): Linear(in_features=10, out_features=10, bias=True)
      (Lambda): Linear(in_features=10, out_features=10, bias=False)
    )
    (5): ELU(alpha=1.0)
  )
  (rho): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ELU(alpha=1.0, inplace=True)
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ELU(alpha=1.0, inplace=True)
    (4): Rho(
      (sig): Sigmoid()
      (fc_mu_1): Linear(in_features=5, out_features=5, bias=True)
      (fc_mu_2): Linear(in_features=

  1
  2
  3
  4
  5
  6
  7
  8
  9
  10
  11
  12
  13
  14
  15
  16
  17
  18
  19
PolicyGeneral(
  (phi): Sequential(
    (0): PermEqui2_max(
      (Gamma): Linear(in_features=2, out_features=20, bias=True)
      (Lambda): Linear(in_features=2, out_features=20, bias=False)
    )
    (1): Tanh()
    (2): PermEqui2_max(
      (Gamma): Linear(in_features=20, out_features=20, bias=True)
      (Lambda): Linear(in_features=20, out_features=20, bias=False)
    )
    (3): Tanh()
    (4): PermEqui2_max(
      (Gamma): Linear(in_features=20, out_features=20, bias=True)
      (Lambda): Linear(in_features=20, out_features=20, bias=False)
    )
    (5): Tanh()
  )
  (rho): Sequential(
    (0): Linear(in_features=20, out_features=5, bias=True)
    (1): ELU(alpha=1.0, inplace=True)
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ELU(alpha=1.0, inplace=True)
    (4): Rho(
      (sig): Sigmoid()
      (fc_mu_1): Linear(in_features=5, out_features=5, bias=True)
      (fc_mu_2): Li

In [ ]:
np.savez_compressed("../results/general_perf_deep_set_arch_loss_50_50", losses_50_50=losses_deep_set)
np.savez_compressed("../results/general_perf_deep_set_arch_test_losses_50_50", test_losses_50_50=test_losses_deep_set)

### Performance vs architecture Program(rho)

In [5]:
import torch 
import torch.nn.functional as F

In [6]:
class RhoReLU(nn.Module):
    def __init__(self, 
            n_steps: int, 
            hidden: int, 
            in_dim: int = 2, 
            sigma_max: float = .3, 
            sigma_min: float = .1
        ) -> None:
        """
        Constructor for PolicyTime torch Module.

        Parameters
        ----------
        n_steps: int
            Number of steps for piece-wise constant solvent strength program.
        hidden: int
            Number of nodes for the hidden layers
        in_dim: int
            length of the encoded analyte set (embedding), it is the input 
            to this network.
        sigma_min: float
            Minimal standard deviation of the solvent strength search space.
            Default value .0. (max value < 1.0)
        sigma_max: float
            Maximal standard deviation of the solvent strength search space.
            Default value .2. (max value is 1.0)
        """
        super().__init__()
        
        self.n_steps = n_steps
        self.hidden = hidden
        self.sigma_min = sigma_min
        self.sigma_max = sigma_max

        self.sig = nn.Sigmoid()
        self.fc_mu_1 = nn.Linear(in_dim, hidden)
        self.fc_mu_2 = nn.Linear(hidden, n_steps)
        self.fc_sig_1 = nn.Linear(in_dim, hidden)
        self.fc_sig_2 = nn.Linear(hidden, n_steps)
          
    def forward(self, x):
        mu = F.relu(self.fc_mu_1(x))
        sigma = F.relu(self.fc_sig_1(x))
        
        mu = self.sig(self.fc_mu_2(mu)).squeeze(0)
        # limit sigma to be in range (sigma_min; sigma_max)
        sigma = self.sig(self.fc_sig_2(sigma)).squeeze(0) * (self.sigma_max - self.sigma_min) + self.sigma_min
        return mu, sigma

In [7]:
class RhoELU(nn.Module):
    def __init__(self, 
            n_steps: int, 
            hidden: int, 
            in_dim: int = 2, 
            sigma_max: float = .3, 
            sigma_min: float = .1
        ) -> None:
        """
        Constructor for PolicyTime torch Module.

        Parameters
        ----------
        n_steps: int
            Number of steps for piece-wise constant solvent strength program.
        hidden: int
            Number of nodes for the hidden layers
        in_dim: int
            length of the encoded analyte set (embedding), it is the input 
            to this network.
        sigma_min: float
            Minimal standard deviation of the solvent strength search space.
            Default value .0. (max value < 1.0)
        sigma_max: float
            Maximal standard deviation of the solvent strength search space.
            Default value .2. (max value is 1.0)
        """
        super().__init__()
        
        self.n_steps = n_steps
        self.hidden = hidden
        self.sigma_min = sigma_min
        self.sigma_max = sigma_max

        self.sig = nn.Sigmoid()
        self.fc_mu_1 = nn.Linear(in_dim, hidden)
        self.fc_mu_2 = nn.Linear(hidden, n_steps)
        self.fc_sig_1 = nn.Linear(in_dim, hidden)
        self.fc_sig_2 = nn.Linear(hidden, n_steps)
          
    def forward(self, x):
        mu = F.elu(self.fc_mu_1(x))
        sigma = F.elu(self.fc_sig_1(x))
        
        mu = self.sig(self.fc_mu_2(mu)).squeeze(0)
        # limit sigma to be in range (sigma_min; sigma_max)
        sigma = self.sig(self.fc_sig_2(sigma)).squeeze(0) * (self.sigma_max - self.sigma_min) + self.sigma_min
        return mu, sigma

In [8]:
class RhoTanh(nn.Module):
    def __init__(self, 
            n_steps: int, 
            hidden: int, 
            in_dim: int = 2, 
            sigma_max: float = .3, 
            sigma_min: float = .1
        ) -> None:
        """
        Constructor for PolicyTime torch Module.

        Parameters
        ----------
        n_steps: int
            Number of steps for piece-wise constant solvent strength program.
        hidden: int
            Number of nodes for the hidden layers
        in_dim: int
            length of the encoded analyte set (embedding), it is the input 
            to this network.
        sigma_min: float
            Minimal standard deviation of the solvent strength search space.
            Default value .0. (max value < 1.0)
        sigma_max: float
            Maximal standard deviation of the solvent strength search space.
            Default value .2. (max value is 1.0)
        """
        super().__init__()
        
        self.n_steps = n_steps
        self.hidden = hidden
        self.sigma_min = sigma_min
        self.sigma_max = sigma_max

        self.sig = nn.Sigmoid()
        self.fc_mu_1 = nn.Linear(in_dim, hidden)
        self.fc_mu_2 = nn.Linear(hidden, n_steps)
        self.fc_sig_1 = nn.Linear(in_dim, hidden)
        self.fc_sig_2 = nn.Linear(hidden, n_steps)
          
    def forward(self, x):
        mu = torch.tanh(self.fc_mu_1(x))
        sigma = torch.tanh(self.fc_sig_1(x))
        
        mu = self.sig(self.fc_mu_2(mu)).squeeze(0)
        # limit sigma to be in range (sigma_min; sigma_max)
        sigma = self.sig(self.fc_sig_2(sigma)).squeeze(0) * (self.sigma_max - self.sigma_min) + self.sigma_min
        return mu, sigma

In [10]:
# Parameters
all_analytes = pd.concat(alists, sort=True).reset_index()[['k0', 'S', 'lnk0']]
activations = [nn.ELU, nn.ReLU, nn.Tanh]
width = [5, 10, 20]
Rhos = [RhoELU, RhoReLU, RhoTanh]
kwargs = {
    'num_episodes' : 25_000, 
    'sample_size' : 10,
    'batch_size' : 1, 
    'lr' : .05, 
    'optim' : torch.optim.SGD,
    'lr_decay_factor' : 0.75,
    'lr_milestones' : 5000,
    'print_every' : 25_001,
    'baseline' : .55,
    'max_norm' : 1.5,
    'max_rand_analytes' : 40,
    'min_rand_analytes' : 8,
    'rand_prob' : 1.,
    'h' : 0.001,
    'run_time' : 1.
}
N = 9
M = 20

losses_rho = np.zeros((N, M, kwargs['num_episodes']))
test_losses_rho = np.zeros((N, M, kwargs['num_episodes']))

delta_taus = np.ones(10) * 1/(10)
for i in range(N):
    print(f"{i}")
    for m in range(M):
        alist_train = all_analytes.sample(frac=0.5)
        alist_test = all_analytes.loc[lambda a: ~a.index.isin(alist_train.index.values)]
        print(f"  {m}")
        #Policies
        pol_50_50 = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
            ),
            rho = nn.Sequential(
                nn.Linear(5, width[i % 3]),
                activations[i // 3](),
                nn.Linear(width[i % 3], width[i % 3]),
                activations[i // 3](),
                Rhos[i // 3](n_steps=len(delta_taus), hidden=width[i % 3], in_dim=width[i % 3], sigma_max=.3, sigma_min=.01),
            )
        )
        
        # Run Exp
        loss, loss_test = reinforce_gen(
            alists = [alist_train], 
            test_alist = alist_test,
            policy = pol_50_50, 
            delta_taus = delta_taus, 
            **kwargs
        )

        losses_rho[i, m] = loss
        test_losses_rho[i, m] = loss_test

0
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
  10
  11
  12
  13
  14
  15
  16
  17
  18
  19
PolicyGeneral(
  (phi): Sequential(
    (0): PermEqui2_max(
      (Gamma): Linear(in_features=2, out_features=5, bias=True)
      (Lambda): Linear(in_features=2, out_features=5, bias=False)
    )
    (1): ELU(alpha=1.0, inplace=True)
    (2): PermEqui2_max(
      (Gamma): Linear(in_features=5, out_features=5, bias=True)
      (Lambda): Linear(in_features=5, out_features=5, bias=False)
    )
    (3): ELU(alpha=1.0, inplace=True)
    (4): PermEqui2_max(
      (Gamma): Linear(in_features=5, out_features=5, bias=True)
      (Lambda): Linear(in_features=5, out_features=5, bias=False)
    )
    (5): ELU(alpha=1.0, inplace=True)
  )
  (rho): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ELU(alpha=1.0)
    (4): RhoELU(
      (sig): Sigmoid()
      (fc_mu_1): Linear(in_features=5, out_featur

  8
  9
  10
  11
  12
  13
  14
  15
  16
  17
  18
  19
PolicyGeneral(
  (phi): Sequential(
    (0): PermEqui2_max(
      (Gamma): Linear(in_features=2, out_features=5, bias=True)
      (Lambda): Linear(in_features=2, out_features=5, bias=False)
    )
    (1): ELU(alpha=1.0, inplace=True)
    (2): PermEqui2_max(
      (Gamma): Linear(in_features=5, out_features=5, bias=True)
      (Lambda): Linear(in_features=5, out_features=5, bias=False)
    )
    (3): ELU(alpha=1.0, inplace=True)
    (4): PermEqui2_max(
      (Gamma): Linear(in_features=5, out_features=5, bias=True)
      (Lambda): Linear(in_features=5, out_features=5, bias=False)
    )
    (5): ELU(alpha=1.0, inplace=True)
  )
  (rho): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Tanh()
    (4): RhoTanh(
      (sig): Sigmoid()
      (fc_mu_1): Linear(in_features=10, out_features=10, bias=True)
      (fc_mu_2): Linear(in_

In [ ]:
np.savez_compressed("../results/general_perf_rho_arch_loss_50_50", losses_50_50=losses_rho)
np.savez_compressed("../results/general_perf_rho_arch_test_losses_50_50", test_losses_50_50=test_losses_rho)

In [ ]:
losses_50_50 = np.load('../results/general_perf_vs_n_steps_losses_50.npz')['losses_50_50']
test_losses_50_50 = np.load('../results/general_perf_vs_n_steps_test_losses_50_50.npz')['test_losses_50_50']
losses_100 = np.load('../results/general_perf_vs_n_steps_losses_100.npz')['losses_100']

In [ ]:
data_100  = []
for i in range(10):
    data_100.append([])
    for j in range(20):
        data_100[i].append(average_over_equal_intervals(losses_100[i,j], 500))
data_100 = np.array(data_100)

data_50_50  = []
for i in range(10):
    data_50_50.append([])
    for j in range(20):
        data_50_50[i].append(average_over_equal_intervals(losses_50_50[i,j], 500))
data_50_50 = np.array(data_50_50)

data_50_50_t  = []
for i in range(10):
    data_50_50_t.append([])
    for j in range(20):
        data_50_50_t[i].append(average_over_equal_intervals(test_losses_50_50[i,j], 500))
data_50_50_t = np.array(data_50_50_t)

In [ ]:
from matplotlib import gridspec
import seaborn as sns
sns.set(style="whitegrid")
sns.set(font_scale=1.3)
gs = gridspec.GridSpec(1, 2, width_ratios=[4, 1]) 

func = np.mean
D = data_100

plt.subplot(gs[0])
plt.rcParams['figure.figsize'] = (15, 10)
for i in range(10):

    plt.plot(np.linspace(0, 25000, 50),func(D, 1)[i], label="nr_steps: "+str(i+1), linewidth=2.)
    #plt.plot(np.linspace(0, 25000, 50),data_50_50[i], label="Train"+str(i+1))
    
plt.legend()
plt.xlabel("Number of episodes")
plt.ylabel("Loss")
plt.subplot(gs[1])
#plt.rcParams['figure.figsize'] = (5, 5)
for i in range(10):

    plt.plot(np.linspace(10000, 25000, 10), func(D, 1)[i][40:], label="nr_steps: "+str(i+1), linewidth=2.)
    #plt.plot(np.linspace(0, 25000, 50),data_50_50[i], label="Train"+str(i+1))
#plt.legend()
plt.xlabel("Number of episodes")
plt.ylabel("Loss")

plt.suptitle("Train Loss(no test set). Each nr_steps is the mean of 20 runs.", fontsize=14)
plt.savefig("train_100_loss_mean.png")

In [ ]:
plt.plot(np.linspace(0, 25000, 50),data_100[9].T, color="r")
plt.plot(np.linspace(0, 25000, 50),data_100[4].T, color="b")
plt.plot(np.linspace(0, 25000, 50),data_100[0].T, color="g")
plt.plot(np.linspace(0, 25000, 50),data_100[0,0], color="g", label="nr_steps: 1")
plt.plot(np.linspace(0, 25000, 50),data_100[4,0], color="b", label="nr_steps: 5")
plt.plot(np.linspace(0, 25000, 50),data_100[9,0], color="r", label="nr_steps: 10")
plt.xlabel("Number of episodes")
plt.ylabel("Loss")
plt.ylim((0, 1.8))
plt.legend()
plt.title("Loss(no test set) for the 20 runs for each nr_steps.", fontsize=14)
plt.savefig("losses_20_data_100.png")

In [ ]:
plt.plot(np.linspace(0, 25000, 50),data_50_50[9].T, color="r")
plt.plot(np.linspace(0, 25000, 50),data_50_50[4].T, color="b")
plt.plot(np.linspace(0, 25000, 50),data_50_50[0].T, color="g")
plt.plot(np.linspace(0, 25000, 50),data_50_50[0,0], color="g", label="nr_steps: 1")
plt.plot(np.linspace(0, 25000, 50),data_50_50[4,0], color="b", label="nr_steps: 5")
plt.plot(np.linspace(0, 25000, 50),data_50_50[9,0], color="r", label="nr_steps: 10")
plt.xlabel("Number of episodes")
plt.ylabel("Loss")
plt.ylim((0, 1.8))
plt.legend()
plt.title("Loss(50/50 train/test split) for the 20 runs for each nr_steps.", fontsize=14)
plt.savefig("losses_20_data_50_50.png")